## _*Using PySCF driver*_

Qiskit Chemistry supports a number of different chemistry drivers, i.e chemistry programs and software libraries, which are used to compute integrals that are then used to build the second quantized Hamiltonian in the FermionicOperator.

Drivers include Gaussian 16, PyQuante, PySCF, PSI4 and HDF5. The main Qiskit documentation has more information on [drivers](https://qiskit.org/documentation/aqua/chemistry/qiskit_chemistry_drivers.html).

For non-Windows platforms (where PySCF has no pre-built packages), the PySCF driver is installed as a  dependent when you `pip install qiskit-aqua`. HDF5 support is built into Qiskit Aqua. If you would like/prefer to use one of the other drivers then refer to the above link for installation and usage guidance.

Note: drivers were written to allow existing users of them to leverage creating the molecular input in a native way for the driver. While Multiplicity (2S+1) is commonly used to specify the overall spin of the molecule, PySCF uses Spin (2S) if you are programming directly with its API and that is what is exposed here. For a singlet system, as in the example below i.e. equal numbers of alpha and beta electrons, the overall spin here is 0 and 2S is 0 (Multiplicity would have been 1).

This notebook has been written to use the PySCF chemistry driver.

In [1]:
import numpy as np

from qiskit import BasicAer

from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQE, NumPyMinimumEigensolver
from qiskit.aqua.components.optimizers import L_BFGS_B
from qiskit.circuit.library import TwoLocal

from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.drivers import PySCFDriver, UnitsType

In [2]:
# using driver to get fermionic Hamiltonian
# PySCF example
driver = PySCFDriver(atom='H .0 .0 .0; H .0 .0 0.735', unit=UnitsType.ANGSTROM,
                     charge=0, spin=0, basis='sto3g')
molecule = driver.run()

In [3]:
# get fermionic operator and mapping to qubit operator
ferOp = FermionicOperator(h1=molecule.one_body_integrals, h2=molecule.two_body_integrals)
qubitOp = ferOp.mapping(map_type='JORDAN_WIGNER', threshold=0.00000001)
qubitOp.chop(10**-10)

In [4]:
# Using exact eigensolver to get the smallest eigenvalue
exact_eigensolver = NumPyMinimumEigensolver(qubitOp)
ret = exact_eigensolver.run()
print('The exact ground state energy is: {}'.format(ret.eigenvalue.real))

The exact ground state energy is: -1.8572750302023824


In [5]:
# setup VQE 
# setup optimizer, use L_BFGS_B optimizer for example
lbfgs = L_BFGS_B(maxfun=1000, factr=10, iprint=10)

# setup variational form generator (generate trial circuits for VQE)
var_form = TwoLocal(qubitOp.num_qubits, 'ry', 'cz', reps=5, entanglement='linear')

# setup VQE with operator, variational form, and optimizer
vqe_algorithm = VQE(qubitOp, var_form, lbfgs)

backend = BasicAer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend)

results = vqe_algorithm.run(quantum_instance)
print("Minimum value: {}".format(results.eigenvalue.real))
print("Parameters: {}".format(results.optimal_point))

Minimum value: -1.85727503015946
Parameters: [ 1.38109648  3.0799131   0.87875453  1.95614111  2.64747338  2.33717554
 -3.47335053  6.71224521  3.55015004 -4.56000624  2.14531307  1.53903717
  1.55226763 -2.61425629 -4.63381128  4.30440231  2.49559166  6.65245959
  4.42675124 -3.74108666  4.93775431  5.66278263  2.00123821 -3.9869573 ]
